In [2]:
%pip install tensorflow scikit-learn seaborn matplotlib

     ------------------------------------ 294.9/294.9 kB 173.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install openpyxl

     ------------------------------------ 250.9/250.9 kB 149.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
import os
from concurrent.futures import ThreadPoolExecutor
import time
import random

# Configuration
EXCEL_FILE = 'sheet.xlsx'
BASE_URL = 'http://52.168.179.101/apg_image/headquarter/'
DATASET_DIR = 'dataset'

TOTAL_IMAGES = 2000
MAX_WORKERS = 10

# Split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# Create dataset folders
train_dir = os.path.join(DATASET_DIR, "train")
val_dir = os.path.join(DATASET_DIR, "val")
test_dir = os.path.join(DATASET_DIR, "test")

for d in [train_dir, val_dir, test_dir]:
    os.makedirs(d, exist_ok=True)


def download_image(args):
    image_name, output_dir = args

    try:
        url = f"{BASE_URL}{image_name}"
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            file_prefix = image_name.split('_')[0] if '_' in image_name else image_name
            file_path = os.path.join(output_dir, f"{file_prefix}.png")

            with open(file_path, 'wb') as file:
                file.write(response.content)

            return True, image_name
        else:
            print(f"Failed: {image_name} ({response.status_code})")
            return False, image_name

    except Exception as e:
        print(f"Error: {image_name} -> {e}")
        return False, image_name


def main():
    try:
        df = pd.read_excel(EXCEL_FILE, sheet_name='Sheet1')
    except Exception as e:
        print(f"Excel error: {e}")
        return

    image_names = df['Image Name'].dropna().unique().tolist()

    if not image_names:
        print("No image names found.")
        return

    print(f"Total found: {len(image_names)}")

    # Limit to 2000
    image_names = image_names[:TOTAL_IMAGES]

    # Shuffle for random split
    random.shuffle(image_names)

    # Compute splits
    total = len(image_names)
    train_end = int(total * TRAIN_RATIO)
    val_end = train_end + int(total * VAL_RATIO)

    train_images = image_names[:train_end]
    val_images = image_names[train_end:val_end]
    test_images = image_names[val_end:]

    print(f"Train: {len(train_images)}, Val: {len(val_images)}, Test: {len(test_images)}")

    # Prepare download tasks
    tasks = (
        [(img, train_dir) for img in train_images] +
        [(img, val_dir) for img in val_images] +
        [(img, test_dir) for img in test_images]
    )

    start_time = time.time()

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(executor.map(download_image, tasks))

    success_count = sum(1 for r in results if r[0])

    print("\nDownload completed!")
    print(f"Downloaded {success_count}/{len(tasks)}")
    print(f"Time: {time.time() - start_time:.2f}s")

    failed = [name for success, name in results if not success]
    if failed:
        with open("failed_downloads.txt", "w") as f:
            f.write("\n".join(failed))
        print("Failed list saved.")


if __name__ == '__main__':
    main()


Total found: 26608
Train: 1400, Val: 400, Test: 200
Error: 609_0.24478180162880647 -> HTTPConnectionPool(host='192.168.1.51', port=8090): Max retries exceeded with url: /ips/block/webcat?cat=83&pl=0&lu=0&url=aHR0cDovLzUyLjE2OC4xNzkuMTAxL2FwZ19pbWFnZS9oZWFkcXVhcnRlci82MDlfMC4yNDQ3ODE4MDE2Mjg4MDY0Nw~~ (Caused by ConnectTimeoutError(<HTTPConnection(host='192.168.1.51', port=8090) at 0x24b16123280>, 'Connection to 192.168.1.51 timed out. (connect timeout=10)'))
Error: 583_0.8620113000082098 -> HTTPConnectionPool(host='192.168.1.51', port=8090): Max retries exceeded with url: /ips/block/webcat?cat=83&pl=0&lu=0&url=aHR0cDovLzUyLjE2OC4xNzkuMTAxL2FwZ19pbWFnZS9oZWFkcXVhcnRlci81ODNfMC44NjIwMTEzMDAwMDgyMDk4 (Caused by ConnectTimeoutError(<HTTPConnection(host='192.168.1.51', port=8090) at 0x24b16123940>, 'Connection to 192.168.1.51 timed out. (connect timeout=10)'))
Error: 549_0.6114094757927416 -> HTTPConnectionPool(host='192.168.1.51', port=8090): Max retries exceeded with url: /ips/block/webcat

In [3]:
"""
============================================================
  FACE RECOGNITION USING ENSEMBLE + TRANSFER LEARNING
  Framework: TensorFlow / Keras
============================================================
  Techniques Used:
  - Transfer Learning: VGG16, ResNet50, MobileNetV2 (pretrained on ImageNet)
  - Ensemble Learning: Soft Voting (average of probabilities)
  - Fine-tuning: Unfreeze top layers of each base model
  - Data Augmentation for robustness
============================================================
"""

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ─────────────────────────────────────────────
#  1. CONFIGURATION
# ─────────────────────────────────────────────
IMAGE_SIZE    = (160, 160)     # Input image size (H x W)
BATCH_SIZE    = 32
EPOCHS_FROZEN = 10             # Epochs while base model is frozen
EPOCHS_FINETUNE = 10           # Epochs after unfreezing top layers
LEARNING_RATE = 1e-4
FINETUNE_LR   = 1e-5           # Lower LR for fine-tuning
NUM_CLASSES   = 5              # ← Change to your number of face classes

# Dataset paths (update these to your directory structure)
# Structure expected:
#   dataset/
#     train/  class_A/  class_B/ ...
#     val/    class_A/  class_B/ ...
#     test/   class_A/  class_B/ ...
TRAIN_DIR = "dataset/train"
VAL_DIR   = "dataset/val"
TEST_DIR  = "dataset/test"

MODELS_SAVE_DIR = "saved_models"
os.makedirs(MODELS_SAVE_DIR, exist_ok=True)


# ─────────────────────────────────────────────
#  2. DATA AUGMENTATION & LOADING
# ─────────────────────────────────────────────
def get_data_generators():
    """Create augmented data generators for train/val/test."""

    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.1,
        zoom_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode="nearest"
    )

    val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_gen = train_datagen.flow_from_directory(
        TRAIN_DIR, target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True
    )
    val_gen = val_test_datagen.flow_from_directory(
        VAL_DIR, target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
    )
    test_gen = val_test_datagen.flow_from_directory(
        TEST_DIR, target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
    )

    return train_gen, val_gen, test_gen


# ─────────────────────────────────────────────
#  3. TRANSFER LEARNING BASE MODELS
# ─────────────────────────────────────────────
def build_transfer_model(base_arch: str, num_classes: int, input_shape=(160, 160, 3)):
    """
    Builds a transfer learning model with a custom classification head.

    Args:
        base_arch: One of 'vgg16', 'resnet50', 'mobilenetv2'
        num_classes: Number of face identity classes
        input_shape: Input image shape (H, W, C)

    Returns:
        Keras Model
    """

    # ── Load pre-trained base (ImageNet weights, exclude top layers) ──
    arch_map = {
        "vgg16":       VGG16,
        "resnet50":    ResNet50,
        "mobilenetv2": MobileNetV2,
    }
    assert base_arch in arch_map, f"Unknown architecture: {base_arch}"

    base_model = arch_map[base_arch](
        weights="imagenet",
        include_top=False,
        input_shape=input_shape
    )

    # ── Freeze all base layers initially ──
    base_model.trainable = False

    # ── Build custom head ──
    inputs = tf.keras.Input(shape=input_shape, name=f"{base_arch}_input")
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name=f"{base_arch}_gap")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation="relu", name=f"{base_arch}_fc1")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation="relu", name=f"{base_arch}_fc2")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax", name=f"{base_arch}_output")(x)

    model = models.Model(inputs, outputs, name=f"FaceRec_{base_arch.upper()}")
    return model, base_model


# ─────────────────────────────────────────────
#  4. FINE-TUNING UTILITY
# ─────────────────────────────────────────────
def unfreeze_top_layers(base_model, num_layers_to_unfreeze: int):
    """
    Unfreezes the last N layers of the base model for fine-tuning.
    All earlier layers remain frozen (preserve low-level ImageNet features).
    """
    base_model.trainable = True
    for layer in base_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False
    print(f"  → Unfroze last {num_layers_to_unfreeze} layers of {base_model.name}")


# ─────────────────────────────────────────────
#  5. TRAINING A SINGLE MODEL (PHASE 1 + 2)
# ─────────────────────────────────────────────
def train_model(arch_name: str, train_gen, val_gen, num_classes: int):
    """
    Full two-phase training:
      Phase 1 - Train custom head only (base frozen)
      Phase 2 - Fine-tune top layers of base model
    """
    print(f"\n{'='*55}")
    print(f"  Training Model: {arch_name.upper()}")
    print(f"{'='*55}")

    model, base_model = build_transfer_model(arch_name, num_classes)
    model.summary()

    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
        ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-7, verbose=1),
        ModelCheckpoint(
            filepath=os.path.join(MODELS_SAVE_DIR, f"best_{arch_name}.keras"),
            save_best_only=True, monitor="val_accuracy", verbose=1
        )
    ]

    # ── Phase 1: Train head only ──
    print(f"\n[Phase 1] Training classification head — base model frozen")
    model.compile(
        optimizer=optimizers.Adam(LEARNING_RATE),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    history1 = model.fit(
        train_gen, validation_data=val_gen,
        epochs=EPOCHS_FROZEN, callbacks=callbacks
    )

    # ── Phase 2: Fine-tune top layers ──
    print(f"\n[Phase 2] Fine-tuning top layers of {arch_name.upper()}")
    unfreeze_layers = {"vgg16": 4, "resnet50": 15, "mobilenetv2": 20}
    unfreeze_top_layers(base_model, unfreeze_layers.get(arch_name, 10))

    model.compile(
        optimizer=optimizers.Adam(FINETUNE_LR),   # Lower LR is critical!
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    history2 = model.fit(
        train_gen, validation_data=val_gen,
        epochs=EPOCHS_FINETUNE, callbacks=callbacks
    )

    # Load best checkpoint
    model.load_weights(os.path.join(MODELS_SAVE_DIR, f"best_{arch_name}.keras"))
    print(f"\n✔ {arch_name.upper()} training complete.\n")

    return model, history1, history2


# ─────────────────────────────────────────────
#  6. ENSEMBLE MODEL (SOFT VOTING)
# ─────────────────────────────────────────────
def build_ensemble(models_list: list, input_shape=(160, 160, 3)):
    """
    Combines multiple models using Soft Voting:
    Final prediction = average of all models' output probabilities.
    Each model gets equal weight (can be customized).
    """
    inputs = tf.keras.Input(shape=input_shape, name="ensemble_input")

    # Collect softmax outputs from each model
    outputs = [model(inputs) for model in models_list]

    # Average the probabilities (soft voting)
    if len(outputs) > 1:
        ensemble_output = layers.Average(name="soft_vote_avg")(outputs)
    else:
        ensemble_output = outputs[0]

    ensemble_model = models.Model(
        inputs=inputs,
        outputs=ensemble_output,
        name="FaceRec_Ensemble"
    )
    return ensemble_model


# ─────────────────────────────────────────────
#  7. EVALUATION UTILITIES
# ─────────────────────────────────────────────
def evaluate_model(model, test_gen, model_name="Model"):
    """Full evaluation: accuracy, classification report, confusion matrix."""
    print(f"\n📊 Evaluating: {model_name}")
    loss, acc = model.evaluate(test_gen, verbose=0)
    print(f"   Test Loss     : {loss:.4f}")
    print(f"   Test Accuracy : {acc*100:.2f}%")

    # Predictions
    test_gen.reset()
    y_pred_probs = model.predict(test_gen, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_gen.classes
    class_names = list(test_gen.class_indices.keys())

    print("\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    return y_true, y_pred, class_names, acc


def plot_confusion_matrix(y_true, y_pred, class_names, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=class_names, yticklabels=class_names
    )
    plt.title(title, fontsize=14, fontweight="bold")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ', '_')}.png", dpi=150)
    plt.show()
    print(f"  Saved: {title.replace(' ', '_')}.png")


def plot_training_history(history1, history2, arch_name):
    """Plot combined training accuracy/loss across both phases."""
    acc  = history1.history["accuracy"]  + history2.history["accuracy"]
    val_acc = history1.history["val_accuracy"] + history2.history["val_accuracy"]
    loss = history1.history["loss"]      + history2.history["loss"]
    val_loss = history1.history["val_loss"]    + history2.history["val_loss"]
    split = len(history1.history["accuracy"])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f"{arch_name.upper()} — Training History", fontsize=14, fontweight="bold")

    ax1.plot(acc, label="Train Acc")
    ax1.plot(val_acc, label="Val Acc")
    ax1.axvline(split, color="gray", linestyle="--", label="Fine-tuning starts")
    ax1.set_title("Accuracy"); ax1.legend(); ax1.set_xlabel("Epoch")

    ax2.plot(loss, label="Train Loss")
    ax2.plot(val_loss, label="Val Loss")
    ax2.axvline(split, color="gray", linestyle="--", label="Fine-tuning starts")
    ax2.set_title("Loss"); ax2.legend(); ax2.set_xlabel("Epoch")

    plt.tight_layout()
    plt.savefig(f"history_{arch_name}.png", dpi=150)
    plt.show()


# ─────────────────────────────────────────────
#  8. PREDICT A SINGLE IMAGE
# ─────────────────────────────────────────────
def predict_face(ensemble_model, image_path: str, class_names: list):
    """
    Predict the identity of a single face image.

    Args:
        ensemble_model: Trained ensemble model
        image_path: Path to input face image
        class_names: List of class labels

    Returns:
        predicted_class (str), confidence (float)
    """
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)   # Shape: (1, H, W, 3)

    preds = ensemble_model.predict(img_array, verbose=0)[0]
    pred_idx = np.argmax(preds)
    confidence = preds[pred_idx]

    print(f"\n🔍 Prediction: {class_names[pred_idx]}  (confidence: {confidence*100:.1f}%)")

    # Show top-3 predictions
    top3_idx = np.argsort(preds)[::-1][:3]
    print("   Top-3 predictions:")
    for i, idx in enumerate(top3_idx):
        print(f"     {i+1}. {class_names[idx]}: {preds[idx]*100:.1f}%")

    return class_names[pred_idx], float(confidence)


# ─────────────────────────────────────────────
#  9. MAIN PIPELINE
# ─────────────────────────────────────────────
def main():
    # ── GPU check ──
    gpus = tf.config.list_physical_devices("GPU")
    print(f"GPUs available: {len(gpus)}")
    if gpus:
        tf.config.experimental.set_memory_growth(gpus[0], True)

    # ── Data ──
    print("\n📂 Loading data...")
    train_gen, val_gen, test_gen = get_data_generators()
    print(f"   Classes: {list(train_gen.class_indices.keys())}")
    print(f"   Train samples : {train_gen.samples}")
    print(f"   Val samples   : {val_gen.samples}")
    print(f"   Test samples  : {test_gen.samples}")

    # ── Train individual models ──
    architectures = ["mobilenetv2", "resnet50", "vgg16"]
    trained_models   = []
    all_histories    = []

    for arch in architectures:
        model, h1, h2 = train_model(arch, train_gen, val_gen, NUM_CLASSES)
        trained_models.append(model)
        all_histories.append((arch, h1, h2))
        plot_training_history(h1, h2, arch)

    # ── Build ensemble ──
    print("\n🔗 Building Ensemble Model (Soft Voting)...")
    ensemble = build_ensemble(trained_models, input_shape=(*IMAGE_SIZE, 3))
    ensemble.summary()

    # ── Evaluate each model individually ──
    results = {}
    for arch, model in zip(architectures, trained_models):
        y_true, y_pred, class_names, acc = evaluate_model(
            model, test_gen, model_name=arch.upper()
        )
        results[arch] = acc
        plot_confusion_matrix(y_true, y_pred, class_names, title=f"CM_{arch.upper()}")

    # ── Evaluate ensemble ──
    y_true, y_pred, class_names, acc_ensemble = evaluate_model(
        ensemble, test_gen, model_name="ENSEMBLE"
    )
    results["ensemble"] = acc_ensemble
    plot_confusion_matrix(y_true, y_pred, class_names, title="CM_ENSEMBLE")

    # ── Summary table ──
    print("\n" + "="*40)
    print("  FINAL ACCURACY COMPARISON")
    print("="*40)
    for name, acc in results.items():
        marker = " ← BEST" if acc == max(results.values()) else ""
        print(f"  {name.upper():<15}: {acc*100:.2f}%{marker}")
    print("="*40)

    # ── Save ensemble ──
    ensemble.save(os.path.join(MODELS_SAVE_DIR, "face_recognition_ensemble.keras"))
    print(f"\n✅ Ensemble model saved to: {MODELS_SAVE_DIR}/face_recognition_ensemble.keras")

    # ── Example single-image prediction ──
    # predict_face(ensemble, "test_face.jpg", class_names)

    return ensemble, class_names


if __name__ == "__main__":
    ensemble_model, class_names = main()

GPUs available: 0

📂 Loading data...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/train'